#Data Minining and Machine Learning
##  Task 3 - Hypothesis Testing

In [ ]:
# Hyper-parameters

# Specified range for feature selection
feat_range = (10, 15)

# SVC kernel type and max number of iteration
# -1 is no limit (default)
svm_kern = 'linear'
svm_iter = -1

# Number of units and layers in MLP hidden
# layers (100,) is 1 layer of 100 units
# Max number of iterations, seed for random weighting of
# units and the learn rate.
mlp_layer_sizes = (100,)
mlp_iter = 500
mlp_rand_state = 42
mlp_learn_rate = 0.001

# Number of cross validation folds
n_cross_val = 10

# Significance value for hypothesis testing
alpha = 0.05


In [ ]:
# This block takes the preprocessing block from task 1 (as asked) 
# to use for this task

import numpy as np
from sklearn.preprocessing import MinMaxScaler

train_data = np.load('x_train.npy')
train_labels = np.load('y_train.npy')
test_data = np.load('x_test.npy')
test_labels = np.load('y_test.npy')

scaler = MinMaxScaler(feature_range=feat_range)
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

In [ ]:
# This block initialises an SVC with a linear kernel, trains it and makes
# predictions on the test set.
# It also stores precision, recall and area under curve scores for later 
# evaluation.
# Lastly, a Precision-Recall Curve for all 10 classification is plotted.

import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

svm_classifier = SVC(kernel=svm_kern, max_iter=svm_iter)
svm_classifier.fit(train_data_scaled, train_labels)
svm_predictions = svm_classifier.decision_function(test_data_scaled)

precision = dict()
recall = dict()
auc_score = dict()
for i in range(len(svm_classifier.classes_)):
    precision[i], recall[i], _ = precision_recall_curve(test_labels == i, svm_predictions[:, i])
    auc_score[i] = auc(recall[i], precision[i])

plt.figure()
for i in range(len(svm_classifier.classes_)):
    plt.plot(recall[i], precision[i], label=f'Class {i} (AUC = {auc_score[i]:0.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('SVM Precision-Recall Curve (Per Class)')
plt.legend(loc='lower left')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()


In [ ]:
# This block initialises an MLP, trains it and makes predictions on the test set.
# Like the SVC block it also stores precision, recall and area under curve scores 
# for later evaluation
# It also plots a Precision-Recall Curve for all 10 classifications.

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

mlp_classifier = MLPClassifier(hidden_layer_sizes=mlp_layer_sizes, learning_rate_init=mlp_learn_rate, max_iter=mlp_iter, random_state=mlp_rand_state)
mlp_classifier.fit(train_data_scaled, train_labels)
mlp_predictions = mlp_classifier.predict_proba(test_data_scaled)

precision = dict()
recall = dict()
auc_score = dict()
for i in range(len(mlp_classifier.classes_)):
    precision[i], recall[i], _ = precision_recall_curve(test_labels == i, mlp_predictions[:, i])
    auc_score[i] = auc(recall[i], precision[i])

plt.figure()
for i in range(len(mlp_classifier.classes_)):
    plt.plot(recall[i], precision[i], label=f'Class {i} (AUC = {auc_score[i]:0.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('MLP Precision-Recall Curve (Per Class)')
plt.legend(loc='lower left')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()

In [ ]:
# This block performs Cross-Validation (CV) on both classifiers.
# This is to collect scores to perform an independent T-Test to obtain a ρ-value
# for our hypothesis test.

import numpy as np
from sklearn.model_selection import cross_val_score
from scipy import stats
from scipy.stats import ttest_ind

svm_classifier = SVC(kernel=svm_kern, max_iter=svm_iter)
svm_classifier.fit(train_data_scaled, train_labels)
svm_scores = cross_val_score(svm_classifier, train_data_scaled, train_labels, cv=n_cross_val)

# MLP Classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=mlp_layer_sizes, learning_rate_init=mlp_learn_rate, max_iter=mlp_iter, random_state=mlp_rand_state)
mlp_classifier.fit(train_data_scaled, train_labels)
mlp_scores = cross_val_score(mlp_classifier, train_data_scaled, train_labels, cv=n_cross_val)


#ttest_ind(svm_scores, mlp_scores)

t_statistic, p_value = stats.ttest_ind(svm_scores, mlp_scores)

print("SVM Scores:", svm_scores)
print("Mean SVM Score:", np.mean(svm_scores))
print("MLP Scores:", mlp_scores)
print("Mean MLP Score:", np.mean(mlp_scores))

print("\nT-test Results:")
print("T-statistic:", t_statistic)
print("ρ-value:", p_value)

In [ ]:
# This block simply compares a chosen α to compare the obtained ρ-value from the
# previous block to make our conclusion on our hypothesis test.

if p_value < alpha:
    if np.mean(svm_scores) > np.mean(mlp_scores):
        print("Reject null hypothesis: SVM performs significantly better than MLP.")
    else:
        print("Reject null hypothesis: MLP performs significantly better than SVM.")
    
else:
    print("Fail to reject null hypothesis: No significant difference between SVM and MLP.")


# Markdown Question

What was the hypothesis you rejected in this task?
How does p-value affect your decision?

#### Answer

Say there exists two hypotheses, a null hypothesis (H0) and an alternative hypothesis (H1) where,

* Null Hypothesis (H0): states that there is no significant difference in performance between the SVM and MLP classifiers.

* Alternative Hypothesis (H1): states that there is significant difference in performance between the SVM and MLP classifiers across all performance metrics.

* and an arbitrary α = 0.05 (significance value of 0.05).

Given α = 0.05 or 5 × 10-2 and

ρ = 3.0388433316268643 × 10-6

A run of this program gave a ρ-value 3.0388433316268643 × 10-6 < 5x10-2 which means the null hypothesis is rejected and the alternative hypothesis which is that there is a significant performance difference is accepted. Based on the accuracy values the SVM is 'significantly' better at classifying the elements of the dataset correctly.